In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
#drop columns
application_df.drop(columns=["EIN", "NAME", "STATUS"], inplace=True)

In [3]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
app_uniques = application_df["APPLICATION_TYPE"].value_counts()
application_types_to_replace = app_uniques[app_uniques < 500].index.to_numpy()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

In [4]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
class_uniques = application_df["CLASSIFICATION"].value_counts()
classifications_to_replace = class_uniques[class_uniques < 1800].index.to_numpy()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

In [5]:
bin_edges = [0, 10000, 100000, 1000000, 10000000, float('inf')]  # Define bin edges based on the range
bin_labels = ['Very Low', 'Low', 'Moderate', 'High', 'Very High']  # Define labels for the bins

#bin ask_amt
application_df['ASK_AMT_binned'] = pd.cut(application_df['ASK_AMT'], bins=bin_edges, labels=bin_labels)
application_df.drop(columns=['ASK_AMT'], inplace=True)

In [6]:
application_df.head(10)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL,ASK_AMT_binned
0,T10,Independent,C1000,ProductDev,Association,0,N,1,Very Low
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,1,Moderate
2,T5,CompanySponsored,C3000,ProductDev,Association,0,N,0,Very Low
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,1,Very Low
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,1,Moderate
5,T3,Independent,C1200,Preservation,Trust,0,N,1,Very Low
6,T3,Independent,C1000,Preservation,Trust,100000-499999,N,1,Low
7,T3,Independent,C2000,Preservation,Trust,10M-50M,N,1,High
8,T7,Independent,C1000,ProductDev,Trust,1-9999,N,1,Low
9,T5,CompanySponsored,C3000,ProductDev,Association,0,N,0,Very Low


In [7]:
pd_dummies = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS',
       'ASK_AMT_binned'])
pd_dummies.head(10)

,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_AMT_binned_Very Low,ASK_AMT_binned_Low,ASK_AMT_binned_Moderate,ASK_AMT_binned_High,ASK_AMT_binned_Very High
0,1,False,True,False,False,False,False,False,False,False,...,False,False,False,True,False,True,False,False,False,False
1,1,False,False,False,True,False,False,False,False,False,...,False,False,False,True,False,False,False,True,False,False
2,0,False,False,False,False,False,True,False,False,False,...,False,False,False,True,False,True,False,False,False,False
3,1,False,False,False,True,False,False,False,False,False,...,False,False,False,True,False,True,False,False,False,False
4,1,False,False,False,True,False,False,False,False,False,...,False,False,False,True,False,False,False,True,False,False
5,1,False,False,False,True,False,False,False,False,False,...,False,False,False,True,False,True,False,False,False,False
6,1,False,False,False,True,False,False,False,False,False,...,False,False,False,True,False,False,True,False,False,False
7,1,False,False,False,True,False,False,False,False,False,...,False,False,False,True,False,False,False,False,True,False
8,1,False,False,False,False,False,False,False,True,False,...,False,False,False,True,False,False,True,False,False,False
9,0,False,False,False,False,False,True,False,False,False,...,False,False,False,True,False,True,False,False,False,False


In [8]:
# Split our preprocessed data into our features and target arrays
X = pd_dummies.drop(columns=['IS_SUCCESSFUL'], axis=1)
y = pd_dummies['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [9]:
scaler = StandardScaler()
# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=46))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=100, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3760      
                                                                 
 dense_1 (Dense)             (None, 100)               8100      
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 11961 (46.72 KB)
Trainable params: 11961 (46.72 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5697 - accuracy: 0.7200
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5551 - accuracy: 0.7291
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5518 - accuracy: 0.7298
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5491 - accuracy: 0.7317
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5479 - accuracy: 0.7316
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5459 - accuracy: 0.7331
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5452 - accuracy: 0.7343
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5445 - accuracy: 0.7345
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5439 - accuracy: 0.7344
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5429 - accura

In [12]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5718 - accuracy: 0.7278 - 326ms/epoch - 1ms/step
Loss: 0.5718221664428711, Accuracy: 0.7278134226799011


In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model_2 = tf.keras.models.Sequential()

# First hidden layer
nn_model_2.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=46))

# Second hidden layer
nn_model_2.add(tf.keras.layers.Dense(units=100, activation="relu"))

# 3rd hidden layer
nn_model_2.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn_model_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model_2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                3760      
                                                                 
 dense_4 (Dense)             (None, 100)               8100      
                                                                 
 dense_5 (Dense)             (None, 30)                3030      
                                                                 
 dense_6 (Dense)             (None, 1)                 31        
                                                                 
Total params: 14921 (58.29 KB)
Trainable params: 14921 (58.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
nn_model_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model
fit_model_2 = nn_model_2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5679 - accuracy: 0.7210
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5534 - accuracy: 0.7291
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5498 - accuracy: 0.7311
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5480 - accuracy: 0.7337
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5461 - accuracy: 0.7342
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5453 - accuracy: 0.7342
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5443 - accuracy: 0.7352
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5426 - accuracy: 0.7360
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5424 - accuracy: 0.7361
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5411 - accura

In [15]:
model_loss, model_accuracy = nn_model_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5949 - accuracy: 0.7266 - 363ms/epoch - 1ms/step
Loss: 0.5948549509048462, Accuracy: 0.7266472578048706


In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_model_3 = tf.keras.models.Sequential()

# First hidden layer
nn_model_3.add(tf.keras.layers.Dense(units=30, activation="relu", input_dim=46))

# Second hidden layer
nn_model_3.add(tf.keras.layers.Dense(units=30, activation="relu"))

# 3rd hidden layer
nn_model_3.add(tf.keras.layers.Dense(units=30, activation="relu"))

#4 th higgen layer
nn_model_3.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn_model_3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model_3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 30)                1410      
                                                                 
 dense_8 (Dense)             (None, 30)                930       
                                                                 
 dense_9 (Dense)             (None, 30)                930       
                                                                 
 dense_10 (Dense)            (None, 30)                930       
                                                                 
 dense_11 (Dense)            (None, 1)                 31        
                                                                 
Total params: 4231 (16.53 KB)
Trainable params: 4231 (16.53 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
# compile the model
nn_model_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model
fit_model_3 = nn_model_3.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5775 - accuracy: 0.7137
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5545 - accuracy: 0.7273
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5511 - accuracy: 0.7318
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5489 - accuracy: 0.7315
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5478 - accuracy: 0.7329
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5461 - accuracy: 0.7339
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5454 - accuracy: 0.7338
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5440 - accuracy: 0.7356
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5434 - accuracy: 0.7358
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5432 - accura

In [18]:
# evaluate the model
model_loss, model_accuracy = nn_model_3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5699 - accuracy: 0.7279 - 362ms/epoch - 1ms/step
Loss: 0.5698596239089966, Accuracy: 0.7279300093650818


# Machine-Learning Analysis and Report
- Overview
    - Create a model that will help Alphabet Soup select applicants for funding with the best chance of success in their ventures.
- Results
    - Data Processing
        - Target for the model - "IS_SUCCESSFUL"
        - Features for the model:
            - "APPLICATION_TYPE"
            - "AFFILIATION"
            - "CLASSIFICATION"
            - "USE_CASE"
            - "ORGANIZATION"
            - "STATUS"
            - "INCOME_AMT"
            - "SPECIAL_CONSIDERATIONS"
            - "ASK_AMT"
        - Removed:
            - "NAME"
            - "EIN"
    - Compiling, Traing, and Evaluation
        - Initial Model
            - 3 Total Layers
            - 1st Layer - 80 neurons - RELU
            - 2nd Layer - 30 neurons - RELU
            - 3rd Layer - 1 neuron - SIGMOID
            - Loss: 0.559
            - Accuracy: 0.7296
        - Optimization Model
            - 3 Total Layers
            - 1st Layer - 80 neurons - RELU
            - 2nd Layer - 100 neurons - RELU
            - 3rd Layer - 1 neuron - SIGMOID
            - Loss: 0.572
            - Accuracy: 0.7278
        - Optimization Model 2
            - 3 Total Layers
            - 1st Layer - 80 neurons - RELU
            - 2nd Layer - 100 neurons - RELU
            - 3rd Layer - 30 neurons - RELU
            - 4th Layer - 1 neuron - SIGMOID
            - Loss: 0.595
            - Accuracy: 0.7266
        - Optimization Model 3
            - 3 Total Layers
            - 1st Layer - 30 neurons - RELU
            - 2nd Layer - 30 neurons - RELU
            - 3rd Layer - 30 neurons - RELU
            - 4th Layer - 30 neurons - RELU
            - 5th Layer - 1 neuron - SIGMOID
            - Loss: 0.570
            - Accuracy: 0.7279
    - Summary
        - This deep-learning model was able to achieve, at the highest, an accuracy of 73% with a loss of 56%. Running more than 100 epochs resulted in over-fitting of the training data and resulting in a lower accuracy score with the test data. More categories or meaningful features would probably have resulted in a more accurate model.